In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv("COM7second1522.csv")
dt=data
data.drop(data.columns[[0]],axis=1,inplace=True)
window_size = int(len(data)/(8))
splittrain = []
splittest = [] 
for i in range(0, len(data)-4*window_size,window_size):
    splittrain.append(data[i:i+3*window_size])
    splittest.append(data[i+3*window_size:i+5*window_size])

In [2]:
from scipy.optimize import NonlinearConstraint, Bounds, LinearConstraint
import sympy as sp
w1=sp.symbols('w1')
w2=sp.symbols('w2')
w3=sp.symbols('w3')
w4=sp.symbols('w4')
w5=sp.symbols('w5')
w6=sp.symbols('w6')
w7=sp.symbols('w7')
w8=sp.symbols('w8')
w9=sp.symbols('w9')
w10=sp.symbols('w10')
td=sp.symbols('td')

In [3]:
def const(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    return w1+w2+w3+w4+w5+w6+w7+w8+w9+w10
nlc = NonlinearConstraint(const, 1 ,1)
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint, Bounds

bounds = [[0, 1], [0, 1],[0, 1], [0, 1],[0, 1],[0, 1],[0, 1], [0, 1],[0, 1],[0,1],[0.2,0.5]]

In [4]:
coll,buy_idx,sell_idx=[],[],[]
count1,count2,count3=0,0,0
def func(p):
    global S,B,resl,resfl
    S=0
    B=0
    resl=[]
    resfl=[]
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    mon=100000
    amount=0
    for i in range(0,len(dfo)):
        r = ((w1*sl1[i]+w2*sl2[i]+w3*sl3[i]+w4*sl4[i]+w5*sl5[i]+w6*sl6[i]+w7*sl7[i]+w8*sl8[i]+w9*sl9[i]+w10*sl10[i])/(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10))#w1=sp.symbols('w1')
        '''if i!=0:
            if dfo['adj_close_price'][i]<dfo['adj_close_price'][i-1]*0.8:
                mon+=amount*dfo['adj_close_price'][i]*0.998
                amount=0'''
        if r>td:
            sumall=mon//dfo['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*dfo['adj_close_price'][i]*1.002
            B+=1
        elif r<-td:
            mon+=amount*dfo['adj_close_price'][i]*0.998
            amount=0
            S+=1
        else:
            pass
    ret=mon+amount*dfo['adj_close_price'][len(dfo)-1]
    return -((ret-100000)/1000)
def fobj():
    global sl1,sl2,sl3,sl4,sl5,sl6,sl7,sl8,sl9,sl10,dfo,decision_d_l,bh_test_l,bh_train_l,train_l,test_l
    bh_test_l,bh_train_l,train_l,test_l,td_l,buy_l,sell_l=[],[],[],[],[],[],[]
    global count1,count2,count3,sell_idx,buy_idx
    count1,count2,count3=0,0,0
    for j in range(len(splittrain)):
        dfo=splittrain[j]
        dfo=dfo.reset_index()
        sl1=dfo['volume_prof_signal']
        sl2=dfo['macd_crossover_signal']
        sl3=dfo['psar_adx_signal']
        sl4=dfo['renko_adx_signal']
        sl5=dfo['aroon_kp_signal']
        sl6=dfo['macd_overzero_signal']
        sl7=dfo['ma5_10_signal']
        sl8=dfo['cdc_signal']
        sl9=dfo['bb_rsi_signal']
        sl10=dfo['rsi_adx_signal']
        result = differential_evolution(func, bounds,constraints=(nlc),maxiter=2100,popsize=80,recombination=0.8)
        coll.append([result.x,-result.fun,j])
        print([result.x,-result.fun,j])
        train_l.append(-result.fun)
        a=result.x
        b=j
        td=a[10]
        w=a[0:10]
        df=splittest[b]
        decision_d_l=[]
        il=[]
        
        mon=100000
        amount=0
        sumall=0
        df=df.reset_index()
        sl1=df['volume_prof_signal']
        sl2=df['macd_crossover_signal']
        sl3=df['psar_adx_signal']
        sl4=df['renko_adx_signal']
        sl5=df['aroon_kp_signal']
        sl6=df['macd_overzero_signal']
        sl7=df['ma5_10_signal']
        sl8=df['cdc_signal']
        sl9=df['bb_rsi_signal']
        sl10=df['rsi_adx_signal']

        for i in range(0,len(df)):
            decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
            '''if i!=0:
                if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                        mon+=amount*df['adj_close_price'][i]*0.998
                        amount=0
                        sell_idx.append(i)'''
            if decision_d>td:
                sumall=mon//df['adj_close_price'][i]
                amount+=sumall
                mon-=sumall*df['adj_close_price'][i]*1.002
                count1+=1
                buy_idx.append(i)
            elif decision_d<-td:
                mon+=amount*df['adj_close_price'][i]*0.998
                amount=0
                count2+=1
                sell_idx.append(i)
            else:
                count3+=1
            decision_d_l.append(decision_d)
        il.append(i)
        ret=mon+amount*df['adj_close_price'][len(df)-1]
        stocknum=100000/dfo['adj_close_price'][0]
        bh_test_l.append(((stocknum*(0.998*df['adj_close_price'][len(df)-1])+(100000-stocknum*df['adj_close_price'][0]*1.002))-100000)/1000)
        bh_train_l.append(((stocknum*(0.998*dfo['adj_close_price'][len(dfo)-1])+(100000-stocknum*dfo['adj_close_price'][0]*1.002))-100000)/1000)
        test_l.append((ret-100000)/1000)
        td_l.append(td)
        buy_l.append(count1)
        sell_l.append(count2)
        print("B&H_train :",bh_train_l)
        print("ATS_train buy:{b} sell:{s} return:{r}".format(b = B,s = S,r=train_l))
        print("B&H_test :",bh_test_l)
        print("td :",td_l)
        print("buy:{b} sell:{s}".format(b=buy_l,s=sell_l))
        print("ATS_test return:{r}".format(r=test_l))

In [5]:
fobj() #new

C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.00465594, 0.05132243, 0.07903476, 0.15589342, 0.35173843,
       0.01329   , 0.1037804 , 0.04228618, 0.17280147, 0.02519696,
       0.4394082 ]), 671.9690391079312, 0]
B&H_train : [406.0033051261028]
ATS_train buy:4 sell:3 return:[671.9690391079312]
B&H_test : [100.21389754960808]
td : [0.43940820345438475]
buy:[1] sell:[2]
ATS_test return:[8.764341842250825]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.09626166, 0.0385527 , 0.39645721, 0.00492875, 0.14143363,
       0.00198528, 0.20830469, 0.05597777, 0.04462343, 0.01147488,
       0.27125478]), 231.6488616338873, 1]
B&H_train : [406.0033051261028, 91.52429838098419]
ATS_train buy:5 sell:8 return:[671.9690391079312, 231.6488616338873]
B&H_test : [100.21389754960808, 320.9511719067274]
td : [0.43940820345438475, 0.2712547815472338]
buy:[1, 8] sell:[2, 9]
ATS_test return:[8.764341842250825, 17.7665194654598]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.04955931, 0.09700275, 0.01929561, 0.17253662, 0.13814001,
       0.01653075, 0.06864738, 0.0208496 , 0.28248136, 0.13495659,
       0.23303549]), 190.8513794001685, 2]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493]
ATS_train buy:6 sell:6 return:[671.9690391079312, 231.6488616338873, 190.8513794001685]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375]
td : [0.43940820345438475, 0.2712547815472338, 0.23303549066028098]
buy:[1, 8, 16] sell:[2, 9, 17]
ATS_test return:[8.764341842250825, 17.7665194654598, 85.90882918307112]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.08569596, 0.19574973, 0.2986177 , 0.21186816, 0.03626008,
       0.02874107, 0.02242334, 0.09755916, 0.00723705, 0.01584773,
       0.27365879]), 445.0221443890303, 3]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493, 145.18524066228727]
ATS_train buy:9 sell:5 return:[671.9690391079312, 231.6488616338873, 190.8513794001685, 445.0221443890303]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375, 111.85490484991536]
td : [0.43940820345438475, 0.2712547815472338, 0.23303549066028098, 0.2736587905306943]
buy:[1, 8, 16, 23] sell:[2, 9, 17, 22]
ATS_test return:[8.764341842250825, 17.7665194654598, 85.90882918307112, 44.23792997581104]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.00845922, 0.00963595, 0.00251907, 0.02439968, 0.01122233,
       0.25103173, 0.16191184, 0.07337736, 0.01670514, 0.4407377 ,
       0.20248558]), 442.4200659096184, 4]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493, 145.18524066228727, 353.28456292402274]
ATS_train buy:427 sell:147 return:[671.9690391079312, 231.6488616338873, 190.8513794001685, 445.0221443890303, 442.4200659096184]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375, 111.85490484991536, -41.11545516196906]
td : [0.43940820345438475, 0.2712547815472338, 0.23303549066028098, 0.2736587905306943, 0.20248557797829006]
buy:[1, 8, 16, 23, 126] sell:[2, 9, 17, 22, 98]
ATS_test return:[8.764341842250825, 17.7665194654598, 85.90882918307112, 44.23792997581104, -1.6614164655837957]


In [6]:
fobj()

C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([1.22007102e-04, 3.82975591e-02, 1.44239202e-03, 2.19895228e-02,
       3.17017776e-01, 4.89592263e-03, 1.35331151e-01, 2.50226645e-01,
       2.30359605e-01, 3.17420110e-04, 4.62311960e-01]), 650.5074607555438, 0]
B&H_train : [406.0033051261028]
ATS_train buy:5 sell:4 return:[650.5074607555438]
B&H_test : [100.21389754960808]
td : [0.4623119600629837]
buy:[0] sell:[1]
ATS_test return:[0.0]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.09879389, 0.46948846, 0.06835604, 0.00436993, 0.01214838,
       0.01875809, 0.00562664, 0.05648721, 0.14581041, 0.12016095,
       0.49736443]), 155.22721187830638, 1]
B&H_train : [406.0033051261028, 91.52429838098419]
ATS_train buy:4 sell:4 return:[650.5074607555438, 155.22721187830638]
B&H_test : [100.21389754960808, 320.9511719067274]
td : [0.4623119600629837, 0.49736442732233954]
buy:[0, 2] sell:[1, 4]
ATS_test return:[0.0, 23.92444237471388]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.20334936, 0.02769291, 0.30551288, 0.09618976, 0.03980898,
       0.00323391, 0.04706859, 0.17331183, 0.00274771, 0.10108409,
       0.22138061]), 221.06965778069892, 2]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493]
ATS_train buy:6 sell:4 return:[650.5074607555438, 155.22721187830638, 221.06965778069892]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375]
td : [0.4623119600629837, 0.49736442732233954, 0.2213806100131272]
buy:[0, 2, 9] sell:[1, 4, 10]
ATS_test return:[0.0, 23.92444237471388, 206.80793376906774]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.27841124, 0.01280142, 0.0326778 , 0.03054379, 0.00623968,
       0.39344876, 0.05419707, 0.15922885, 0.00972693, 0.02272444,
       0.43911489]), 273.3899186196575, 3]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493, 145.18524066228727]
ATS_train buy:40 sell:19 return:[650.5074607555438, 155.22721187830638, 221.06965778069892, 273.3899186196575]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375, 111.85490484991536]
td : [0.4623119600629837, 0.49736442732233954, 0.2213806100131272, 0.43911488661521536]
buy:[0, 2, 9, 33] sell:[1, 4, 10, 27]
ATS_test return:[0.0, 23.92444237471388, 206.80793376906774, 60.190703952941895]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.02731506, 0.00776883, 0.31657039, 0.01057321, 0.26600271,
       0.01576337, 0.2943585 , 0.02308925, 0.03803534, 0.00052333,
       0.31074288]), 558.1034982837182, 4]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493, 145.18524066228727, 353.28456292402274]
ATS_train buy:9 sell:8 return:[650.5074607555438, 155.22721187830638, 221.06965778069892, 273.3899186196575, 558.1034982837182]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375, 111.85490484991536, -41.11545516196906]
td : [0.4623119600629837, 0.49736442732233954, 0.2213806100131272, 0.43911488661521536, 0.31074287624228847]
buy:[0, 2, 9, 33, 34] sell:[1, 4, 10, 27, 32]
ATS_test return:[0.0, 23.92444237471388, 206.80793376906774, 60.190703952941895, 2.8301151716308524]


In [7]:
fobj()

C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([2.93631851e-03, 2.59850046e-03, 4.97026986e-02, 1.04111458e-01,
       2.64319039e-01, 9.12879808e-05, 2.62557797e-01, 1.62677173e-01,
       1.39646068e-01, 1.13596588e-02, 4.32829813e-01]), 673.7400156075871, 0]
B&H_train : [406.0033051261028]
ATS_train buy:5 sell:4 return:[673.7400156075871]
B&H_test : [100.21389754960808]
td : [0.4328298130377456]
buy:[0] sell:[3]
ATS_test return:[0.0]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([1.00977231e-01, 1.72879589e-04, 1.24945553e-01, 2.22904927e-01,
       1.38503351e-01, 9.92352609e-03, 9.95014139e-02, 9.32056464e-03,
       1.15880672e-01, 1.77869883e-01, 2.31986851e-01]), 248.83935823262024, 1]
B&H_train : [406.0033051261028, 91.52429838098419]
ATS_train buy:25 sell:6 return:[673.7400156075871, 248.83935823262024]
B&H_test : [100.21389754960808, 320.9511719067274]
td : [0.4328298130377456, 0.23198685095287033]
buy:[0, 21] sell:[3, 5]
ATS_test return:[0.0, 80.4379422834206]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.00846792, 0.00393374, 0.01704857, 0.17986041, 0.07753489,
       0.02846703, 0.2755259 , 0.23286812, 0.00326324, 0.17303017,
       0.31065672]), 258.2952903267269, 2]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493]
ATS_train buy:14 sell:8 return:[673.7400156075871, 248.83935823262024, 258.2952903267269]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375]
td : [0.4328298130377456, 0.23198685095287033, 0.3106567163820871]
buy:[0, 21, 31] sell:[3, 5, 12]
ATS_test return:[0.0, 80.4379422834206, 236.04922733727452]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.12776863, 0.02346176, 0.00157199, 0.0242185 , 0.06863082,
       0.15895936, 0.0097004 , 0.46588108, 0.0257374 , 0.09407005,
       0.36027093]), 266.8813204902915, 3]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493, 145.18524066228727]
ATS_train buy:22 sell:14 return:[673.7400156075871, 248.83935823262024, 258.2952903267269, 266.8813204902915]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375, 111.85490484991536]
td : [0.4328298130377456, 0.23198685095287033, 0.3106567163820871, 0.3602709347947555]
buy:[0, 21, 31, 47] sell:[3, 5, 12, 24]
ATS_test return:[0.0, 80.4379422834206, 236.04922733727452, 58.89794724689488]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.01454819, 0.02382543, 0.03739906, 0.03669507, 0.00372521,
       0.43961907, 0.07713168, 0.06616166, 0.27669133, 0.02420331,
       0.28537898]), 374.4412070865631, 4]
B&H_train : [406.0033051261028, 91.52429838098419, 113.89101365449493, 145.18524066228727, 353.28456292402274]
ATS_train buy:398 sell:132 return:[673.7400156075871, 248.83935823262024, 258.2952903267269, 266.8813204902915, 374.4412070865631]
B&H_test : [100.21389754960808, 320.9511719067274, 537.8456274888375, 111.85490484991536, -41.11545516196906]
td : [0.4328298130377456, 0.23198685095287033, 0.3106567163820871, 0.3602709347947555, 0.2853789790103471]
buy:[0, 21, 31, 47, 153] sell:[3, 5, 12, 24, 147]
ATS_test return:[0.0, 80.4379422834206, 236.04922733727452, 58.89794724689488, 8.279431593463872]


In [ ]:
fobj()

In [ ]:
fobj()

In [27]:
# coll=[[[0.08492177, 0.17384463, 0.01297783, 0.02911209, 0.00305128,
#        0.29679029, 0.09341777, 0.03109278, 0.2669813 , 0.00781026,
#        0.15002345], 77.78606595899961, 0],[[0.10124751, 0.07947271, 0.14059461, 0.01134871, 0.07670483,
#        0.0105102 , 0.01523628, 0.0793311 , 0.32688289, 0.15867116,
#        0.16469997], 29.423729452217085, 1],[[0.10424396, 0.04083642, 0.14153184, 0.04832486, 0.02425143,
#        0.23456657, 0.01785653, 0.04702162, 0.1687655 , 0.17260126,
#        0.23068131], 20.744831571363452, 2],[[0.03418246, 0.03985494, 0.05818124, 0.01006821, 0.05736128,
#        0.51578311, 0.04809931, 0.05745843, 0.05766466, 0.12134635,
#        0.13210764], 17.658730539011014, 3],[[0.04441311, 0.00105711, 0.10547514, 0.24048136, 0.00218894,
#        0.03192681, 0.0093497 , 0.09019205, 0.3072222 , 0.16769358,
#        0.2397252 ], 38.12829733082201, 4],[[0.15608604, 0.46143613, 0.02077728, 0.15747731, 0.00652082,
#        0.03187775, 0.00144456, 0.02996531, 0.12415056, 0.01026426,
#        0.05789345], 34.24385590455824, 5],[[0.03761342, 0.34845239, 0.11776582, 0.00239228, 0.03168897,
#        0.02931779, 0.15102881, 0.08606771, 0.11747511, 0.07819771,
#        0.18479926], 27.365418176465987, 6],[[0.03660642, 0.05033639, 0.12932883, 0.07908549, 0.01884995,
#        0.00878074, 0.15192304, 0.25559357, 0.00167098, 0.2678246 ,
#        0.22253394], 48.53843252603145, 7],[[0.00772096, 0.47030788, 0.07248883, 0.04361143, 0.25529928,
#        0.04781215, 0.00984386, 0.00458237, 0.02302817, 0.06530506,
#        0.11147168], 92.85205689248085, 8],[[0.02404885, 0.11613386, 0.04691288, 0.15394344, 0.01931855,
#        0.0351663 , 0.15083233, 0.02116866, 0.27700484, 0.15547028,
#        0.28969039], 78.26595460097695, 9]]